# Slicing

Objects in scipp can be sliced in two ways. The most common is by by index based slicing. A second, more limited approach is to use value based slicing based on coordinate values.

## Index based slicing

Data in a [Variable](../generated/scipp.Variable.rst#scipp.Variable), [Dataset](../generated/scipp.Dataset.rst#scipp.Dataset) or [DataArray](../generated/scipp.DataArray.rst#scipp.DataArray) can be indexed in a similar manner to NumPy and xarray.
The dimension to be sliced is specified using a dimension label and, in contrast to NumPy, positional dimension lookup is not available.
Positional indexing with an integer or an integer range is made via `__getitem__` and `__setitem__` with a dimension label as first argument.
This is available for Variables, DataArrays, Datasets, as well as items of a Dataset.
In all cases a *view* is returned, i.e., just like when slicing a [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) no copy is performed.

Consider the following variable:

In [ ]:
import numpy as np
import scipp as sc

var = sc.Variable(
    dims=['z', 'y', 'x'],
    values=np.random.rand(2, 3, 4),
    variances=np.random.rand(2, 3, 4))
sc.show(var)

As when slicing a `numpy.ndarray`, the dimension `'x'` is removed since no range is specified:

In [ ]:
s = var['x', 1]
sc.show(s)
print(s.dims, s.shape)

When a range is specified, the dimension is kept, even if it has extent 1:

In [ ]:
s = var['x', 1:3]
sc.show(s)
print(s.dims, s.shape)

s = var['x', 1:2]
sc.show(s)
print(s.dims, s.shape)

Slicing can be chained arbitrarily:

In [ ]:
s = var['x', 1:4]['y', 2]['x', 1]
sc.show(s)
print(s.dims, s.shape)

Slicing for datasets works in the same way, but some additional rules apply:

In [ ]:
d = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2)),
     'c': sc.Variable(dims=['x'], values=np.random.rand(2)),
     '0d-data': sc.Variable(1.0)},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m),
        'aux_x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'aux_y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})
sc.show(d)

As when slicing a variable, the sliced dimension is removed when slicing without range, and kept when slicing with range.

When slicing a dataset a number of other things happen as well:

- Any data item that does not depend on the sliced dimension is removed.
- Slicing **without range**:
  - The *coordinates* for the sliced dimension are *removed*.
- Slicing **with a range**:
  - The *coordinates* for the sliced dimension are *kept*.

The rationale behind this mechanism is as follows.
We may want to modify slices independently, e.g., by adding an offset to certain slices:

In [ ]:
d['x', 0] += 1.0
d['x', 1] += 2.0

By excluding scalar items from the slice view (see below for a visual representation), we prevent unintentional addition of multiple offsets to the same scalar.

This is an important aspect and it is worthwhile to take some time and think through the mechanism.
Consider the following example, contrasting slicing with and without range:

- We slice dimension `'x'`, so the data item `'0d-data'` which does not depend on dimension `'x'` is not visible in the slice views.
- In the second case (without range) the coord for dimension `'x'` is also not part of the slice view

Make sure to inspect the `dims` and `shape` of all variable (data and coordinates) of the resulting slice views (note the tooltip shown when moving the mouse over the name also contains this information):

In [ ]:
# Range of length 1
sc.show(d['x', 1:2])
d['x', 1:2]

In [ ]:
# No range
sc.show(d['x', 1])
d['x', 1]

Slicing a data item of a dataset should not bring any surprises.
Essentially this behaves like slicing a dataset with just a single data item:

In [ ]:
sc.show(d['a']['x', 1:2])

Slicing and item access can be done in arbitrary order with identical results:

In [ ]:
d['x', 1:2]['a'] == d['a']['x', 1:2]
d['x', 1:2]['a'].coords['x'] == d.coords['x']['x', 1:2]

## Value based slicing

Data in a [Dataset](../generated/scipp.Dataset.rst#scipp.Dataset) or [DataArray](../generated/scipp.DataArray.rst#scipp.DataArray) can be sliced by the coordinate value.
Value indexing with a [Variable](../generated/scipp.Variable.rst#scipp.Variable)  or an [Variable](../generated/scipp.Variable.rst#scipp.Variable) range is made via `__getitem__` and `__setitem__` with a dimension label as first argument.
This is available for DataArrays and Datasets, as well as items of a Dataset, but not for Variable types as coordinates are necessary.
In all cases a *view* is returned, i.e., just like when slicing a [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) no copy is performed.



### Value slicing on point coordinates

Lets create a simple `DataArray`. It has dimension coordinates in x and y:

In [ ]:
da = sc.DataArray(data = sc.Variable(dims=['x', 'y'], values=np.arange(8.0).reshape(2, 4)),
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(4.0), unit=sc.units.m)})
sc.show(da)

Lets say we wish to slice the `DataArray` where y coord is exactly at the point 1.0 meters expressed as a variable. You will notice that the data item in the output have no y coordinate dependency after the point slicing operation.

In [ ]:
point_value = 0.0 * sc.units.m
da['y', point_value] 

Our y-coordinate is represents point data rather than bin edges for the dataset data. This means that when slicing with a point value, the value needs to exactly match a coordinate value. If it does not, an `IndexError` is raised.

In [ ]:
point_value = 0.1 * sc.units.m # No y coordinate value at this point
try:
    d['y', point_value]
except IndexError as e:
    print(str(e))

We can also slice a range just like index slicing above. No exact match to coordinate value of start/stop is required in the case of range slicing.

In [ ]:
start = 0.0 * sc.units.m
stop = 1.1 * sc.units.m
d['y', start:stop]

### Value slicing on bin edge coordinates

As mentioned above, value based slicing is valid for `Datasets` and `DataArrays` We construct a simple `DataArray` to illustrate the working. In this `DataArray` the y-coordinate is bin edges.

In [ ]:
da = sc.DataArray(data = sc.Variable(dims=['x', 'y'], values=np.arange(8.0).reshape(2, 4)),
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(5.0), unit=sc.units.m)})
sc.show(da)

Because we have bin edges in the y dimension, each bin represents a range between the defined edges. Therefore point slicing between coordinate values is a valid operation.

In [ ]:
point_value = 0.1 * sc.units.m # select between coord values 0 - 1 meter
da['y', point_value]
lower_edge = 0.0 * sc.units.m # on lower edge
assert sc.is_equal(da['y', point_value], da['y', lower_edge])

Range based slicing is also valid on bin edge coordinates.

In [ ]:
start = 1.1 * sc.units.m
stop = None 
sc.show(da['y', start:stop])

### Value based slicing caveats 

At the time of writing value based slicing is only possible for certain conditions.

* The coordinate selected for slicing must be monotonic, otherwise it will need to be sorted first
* The coordinate cannot be multi-dimensional 